In [1]:
using InteractiveUtils, Test

In [2]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [3]:
pw_rec(0, "Abc"), pw_rec(1, "Abc"), pw_rec(2, "Abc"), pw_rec(3, "Abc")

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [4]:
function pw_cont(n, x::T, k) where {T}
    if iszero(n)
        k(one(T))
    else
        pw_cont(n - 1, x, r -> k(x * r))
    end
end

pw_cont (generic function with 1 method)

In [5]:
pw_cont(n, x) = pw_cont(n, x, identity)

pw_cont (generic function with 2 methods)

In [6]:
pw_cont(0, "Abc"), pw_cont(1, "Abc"), pw_cont(2, "Abc"), pw_cont(3, "Abc")

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [7]:
@inline mult_opt(::Val{true}, x , r) = x
@inline mult_opt(::Val{false}, x , r) = x * r

mult_opt (generic function with 2 methods)

In [8]:
@inline function pw_cont_abs(n, x::T, k) where {T}
    if iszero(n)
        k(Val(true), one(T))
    else
        pw_cont_abs(n - 1, x, (r_one, r) -> k(Val(false), mult_opt(r_one, x, r)))
    end
end

pw_cont_abs (generic function with 1 method)

In [9]:
@inline pw_cont_abs(n, x) = pw_cont_abs(n, x, (r_one, r) -> r)

pw_cont_abs (generic function with 2 methods)

In [10]:
pw_cont_abs(0, "Abc"), pw_cont_abs(1, "Abc"), pw_cont_abs(2, "Abc"), pw_cont_abs(3, "Abc")

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [11]:
Base.iszero(::Val{n}) where {n} = iszero(n)

In [12]:
iszero(Val(0)), iszero(Val(10))

(true, false)

In [13]:
Base.:-(::Val{n}, m) where {n} = Val(n - m)

In [14]:
Val(10) - 3

Val{7}()

In [15]:
Tuple(pw_cont_abs(Val(i), 10) for i in 0:3)

(1, 10, 100, 1000)

In [16]:
@code_typed pw_cont_abs(Val(2), 10)

CodeInfo(
1 ─ %1 = %new(var"#11#12"{Int64, var"#13#14"}, x, var"#13#14"())::var"#11#12"{Int64, var"#13#14"}
│   %2 = invoke Main.pw_cont_abs($(QuoteNode(Val{1}()))::Val{1}, x::Int64, %1::var"#11#12"{Int64, var"#13#14"})::Any
└──      return %2
) => Any

In [17]:
@inline function pw_tail(n, x, r)
    if iszero(n)
        r
    else
        pw_tail(n - 1, x, x * r)
    end
end

pw_tail (generic function with 1 method)

In [18]:
@inline pw_tail(n, x::T) where {T} = pw_tail(n, x, one(T))

pw_tail (generic function with 2 methods)

In [19]:
Tuple(pw_tail(Val(i), "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [20]:
@code_typed pw_tail(Val(3), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, 1)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

In [21]:
@inline function pw_tail_opt(n, x, r_one, r)
    if iszero(n)
        r
    else
        pw_tail_opt(n - 1, x, Val(false), mult_opt(r_one, x, r))
    end
end

pw_tail_opt (generic function with 1 method)

In [22]:
@inline pw_tail_opt(n, x::T) where {T} = pw_tail_opt(n, x, Val(true), one(T))

pw_tail_opt (generic function with 2 methods)

In [23]:
Tuple(pw_tail_opt(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [24]:
Tuple(pw_tail_opt(Val(i), "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [25]:
@code_typed pw_tail_opt(Val(3), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
└──      return %2
) => Int64